In [ ]:
import pandas as pd
from tqdm import tqdm
import string
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import re
import spacy
import copy
import nltk
from nltk.corpus import mac_morpho
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import pickle
import json

In [ ]:
nltk.download('punkt')
nltk.download('mac_morpho')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')
nltk.download('stopwords')

In [ ]:
!pip install unidecode
!python -m spacy download pt_core_news_lg
!pip install git+https://github.com/LIAAD/yake
!pip install git+https://github.com/LIAAD/yake --upgrade
!pip install textacy

In [ ]:
import unidecode
import yake
import textacy
NLP = spacy.load("pt_core_news_lg")

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Itau-Unibanco/FAQ_BACEN")
train_dataset = load_dataset("Itau-Unibanco/FAQ_BACEN", split="train")
train_dataset['categories']
{'Unnamed: 0': Value(dtype='int64', id=None),
 'questions': Value(dtype='string', id=None),
 'categories': Value(dtype='string', id=None),
 'answers': Value(dtype='string', id=None)}

categorias = train_dataset['categories']
questions =  train_dataset['questions']
answers =  train_dataset['answers']

all_data = categorias
for q in questions:
  all_data.append(q)
for q in answers:
  all_data.append(q)

df_data = {'descricao': all_data}
data_shopping = pd.DataFrame(df_data)

generic_words = ['de',  'a',  'o',  'que',  'e',  'do',  'da',  'em',  'um',  'para',  'é',  'com',  'não',  'uma',  'os',  'no',  'se',  'na',  'por',  'mais',  'as',  'dos',  'como',  'mas',  'foi',  'ao',  'ele',  'das',  'tem',  'à',  'seu',  'sua',  'ou',  'ser',  'quando',  'muito',  'há',  'nos',  'já',  'está',  'eu',  'também',  'só',  'pelo',  'pela',  'até',  'isso',  'ela',  'entre',  'era',  'depois',  'sem',  'mesmo',  'aos',  'ter',  'seus',  'quem',  'nas',  'me',  'esse',  'eles',  'estão',  'você',  'tinha',  'foram',  'essa',  'num',  'nem',  'suas',  'meu',  'às',  'minha',  'têm',  'numa',  'pelos',  'elas',  'havia',  'seja',  'qual',  'será',  'nós',  'tenho',  'lhe',  'deles',  'essas',  'esses',  'pelas',  'este',  'fosse',  'dele',  'tu',  'te',  'vocês',  'vos',  'lhes',  'meus',  'minhas',  'teu',  'tua',  'teus',  'tuas',  'nosso',  'nossa',  'nossos',  'nossas',  'dela',  'delas',  'esta',  'estes',  'estas',  'aquele',  'aquela',  'aqueles',  'aquelas',  'isto',  'aquilo',  'estou',  'está',  'estamos',  'estão',  'estive',  'esteve',  'estivemos',  'estiveram',  'estava',  'estávamos',  'estavam',  'estivera',  'estivéramos',  'esteja',  'estejamos',  'estejam',  'estivesse',  'estivéssemos',  'estivessem',  'estiver',  'estivermos',  'estiverem',  'hei',  'há',  'havemos',  'hão',  'houve',  'houvemos',  'houveram',  'houvera',  'houvéramos',  'haja',  'hajamos',  'hajam',  'houvesse',  'houvéssemos',  'houvessem',  'houver',  'houvermos',  'houverem',  'houverei',  'houverá',  'houveremos',  'houverão',  'houveria',  'houveríamos',  'houveriam',  'sou',  'somos',  'são',  'era',  'éramos',  'eram',  'fui',  'foi',  'fomos',  'foram',  'fora',  'fôramos',  'seja',  'sejamos',  'sejam',  'fosse',  'fôssemos',  'fossem',  'for',  'formos',  'forem',  'serei',  'será',  'seremos',  'serão',  'seria',  'seríamos',  'seriam',  'tenho',  'tem',  'temos',  'tém',  'tinha',  'tínhamos',  'tinham',  'tive',  'teve',  'tivemos',  'tiveram',  'tivera',  'tivéramos',  'tenha',  'tenhamos',  'tenham',  'tivesse',  'tivéssemos',  'tivessem',  'tiver',  'tivermos',  'tiverem',  'terei',  'terá',  'teremos',  'terão',  'teria',  'teríamos',  'teriam']

In [ ]:
all_descs_ngrams = []

def filter_preprocess(filter='generic'):
    all_descs_bigrams = []
    sentences = list(data_shopping['descricao'].values)
    stop_words = nltk.corpus.stopwords.words('portuguese')
    filtered_sentences=[]

    #initiate lemmas with the generic_words lemmas
    gen_words_lemmas = []
    for word in generic_words:
        generic_tagger = NLP(word)
        for token in generic_tagger:
            if token.lemma_ not in gen_words_lemmas:
                gen_words_lemmas.append(token.lemma_)

    #preprocess each sentence description
    for sentence in tqdm(sentences):
        text = sentence
        text=unidecode.unidecode(str(text).lower())
        text = re.sub(r'[^a-zA-Z]', ' ', text)
        sentence = (" ".join(text.split()))
        #print(sentence +"\n")

        words = []
        chunks =[]
        s_bigrams = []
        lemmas = []
        lemmas.extend(gen_words_lemmas)
        if len(sentence) > 2:
            tagger = NLP(sentence)
            # looking for bigrams and trigrams in the sentence
            # chunks = list(textacy.extract.ngrams(tagger, 2, min_freq=6))
            # chunks.extend(list(textacy.extract.ngrams(tagger, 3, min_freq=5)))
            # for word in chunks:
            #     word = str(word).replace(" ", "_")
            #     #only save the ngram if it is in the reference list
            #     if (word not in s_bigrams):# and (word in bigramas_ref):
            #         s_bigrams.append(word)
            #print(s_bigrams)

            #filtering some gramatical classes, generic and stop_words
            for token in tagger:
                if token.pos_ not in ['ADV', 'CCONJ', 'ADP', 'AUX', 'CONJ', 'DET', 'INTJ', 'PART', 'PRON', 'PUNCT', 'SYM','SCONJ','ADJ','VERB', 'PROPN']:
                    text = str(token.orth_)
                    #print(text, token.lemma_)
                    if text not in stop_words and text not in generic_words:
                        if token.lemma_ not in lemmas:
                            words.append(text)
                            lemmas.append(token.lemma_)

            #print(lemmas)
            #words.extend(s_bigrams)
            sentence= ' '.join(words)
            #print(sentence)
            #break
        #all_descs_ngrams.append(s_bigrams)
        filtered_sentences.append(sentence)

    return filtered_sentences

#[ADV, CCONJ, ADP, AUX, CONJ, DET, INTJ, PART, PRON, PUNCT, SYM,SCONJ,ADJ,VERB, PROPN]:

In [ ]:
# Se atualizada a lista de palavras genericas, rodar a partir desse ponto
filtered_sentences = []

filtered_sentences = filter_preprocess()

#data_shopping['descricao_processed'] = filtered_sentences

### Extração de palavras chaves com o YAKE!



In [ ]:
def remove_semelhantes(words_list):
    palavras_finais = []
    for word in words_list:
        if word not in palavras_finais:
            palavras_finais.append(word)
    #print(palavras_finais)
    words_list =palavras_finais.copy()
    for t in range(0, len(words_list)-1):
        for i in range(t+1, len(words_list)):
            maxW = max(len(words_list[t]), len(words_list[i]))
            distance=nltk.edit_distance(words_list[t] , words_list[i])
            #print((1- distance/maxW))
            if (1- distance/maxW) > 0.68:
                maxP = max(words_list[t], words_list[i])
                if maxP in palavras_finais:
                    palavras_finais.remove(maxP)
    return palavras_finais

In [ ]:
dict_nodes = {}
nodes = ['bacen']
tipos = ['bacen'] * len(filtered_sentences)
df_yake = {'tipo':tipos,'descricao_processed':filtered_sentences}
data_shopping = pd.DataFrame(df_yake) 
data_shopping.dropna(subset=['descricao_processed']).reset_index(drop=True)

language = "pt"
max_ngram_size = 1
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 40

for node in tqdm(nodes):
    print("\n",node)
    data_node = data_shopping[data_shopping['tipo'] == node]

    indexes = list(data_node.index)
    #print(max(indexes))
    # ngrams_node = []
    # for i in indexes:
    #     ngrams_node.extend(all_descs_ngrams[i])

    # # #print(node+ ":", ngrams_node)
    # ngrams_node = remove_semelhantes(ngrams_node)
    #print(node+ ":", ngrams_node)

    sentences=list(data_node['descricao_processed'].values)

    #Processando as palavras-chaves
    text = ' '.join(map(str, sentences))

    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    words = []
    #values = []
    for kw in keywords:
        #values.append(kw[1])
        words.append(kw[0])

    palavras_finais = []
    palavras_finais = remove_semelhantes(words)

    #palavras_finais.extend(ngrams_node)
    dict_nodes[node] = palavras_finais



In [ ]:
all_words = []
for node in nodes:
    all_words.extend(dict_nodes[node])
    print(node, len(dict_nodes[node]))
    print(dict_nodes[node], '\n')
